In [ ]:
import sys
sys.path.append('../')


In [ ]:
import general_util as gu
from tqdm.auto import tqdm
import math
import torch
import sys
import numpy as np
from simpletransformers.language_representation import RepresentationModel
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.manifold import TSNE


In [ ]:

bert_mean_embedding = RepresentationModel(
            model_type="bert", model_name="bert-base-uncased",
            use_cuda=True)

In [ ]:
#change the system output to save the results in a file
old_stdout = sys.stdout
sys.stdout= open("cls_svm_tsn_pcs_2.txt","w")

In [ ]:
def new_predict(clf, new_bias, X):
    return np.sign(clf.decision_function(X) - new_bias)

In [ ]:
print(torch.cuda.device_count())
gpu_ids = [gpu_id for gpu_id in range(torch.cuda.device_count())]
print(gpu_ids)
device = torch.device(f'cuda:{gpu_ids[0]}')
print(device)
torch.cuda.set_device(device)

In [ ]:
data_path ="../data/"


print("svm with rbf kernel and gamma=0.3 on bert mean embeddings")

for fold_id in [0,1,2,3,4]:
    
    print(f"\n********************fold{fold_id}*******************\n")
    #print(f"this file uses no_answer_threshold: {thresholds[fold_id]}")
    #train
    print("Training............")
    x_train,y_train = gu.get_train_data(fold_id)
    print(len(x_train))
    train_embed = gu.get_bert_embedding(x_train)
    clf = svm.SVC(kernel='rbf',gamma=0.3)
    #clf = svm.LinearSVC()
    clf.fit(train_embed,y_train)
    
    print("Validation....")
    #test
    x_val,y_val = gu.get_val_data(fold_id)
    val_embed = gu.get_bert_embedding(x_val)
    
    
    res = clf.predict(val_embed)
    
    print(classification_report(y_val,res))
    


In [ ]:
#this cell to use svm on simCSE embedding

data_path ="../data/"

print("svm with rbf kernel and gamma=0.5 and simCSE embeddings")

for fold_id in [0,1,2,3,4]:
    
    print(f"\n********************fold{fold_id}*******************\n")
    #print(f"this file uses no_answer_threshold: {thresholds[fold_id]}")
    #train
    print("Training............")
    x_train,y_train = gu.get_train_data(fold_id)
    print(len(x_train))
    train_embed = gu.get_simCSE_embedding(x_train)
    clf = svm.SVC(kernel='rbf',gamma=0.2,C=0.8)
    clf.fit(train_embed,y_train)
    
    print("Validation....")
    #test
    x_val,y_val = gu.get_val_data(fold_id)
    val_embed = gu.get_simCSE_embedding(x_val)
    res = clf.predict(val_embed)
    
    print(classification_report(y_val,res))
    
    #print(res)
    #print(y_val)

In [ ]:
data_path ="../data/"

for fold_id in [0,1,2,3,4]:
    
    print(f"\n********************fold{fold_id}*******************\n")
    
    print("Validation....")
    #test
    x_val,y_val = gu.get_val_data(fold_id)
    print(len(x_val))
    val_embed = gu.get_bert_embedding(x_val)
    new_res = new_predict(clf,0.5,val_embed)
    
    print(classification_report(y_val,new_res))
    
    print(new_res)
    print(y_val)

In [ ]:
#this code uses svm on pca

import sys
old_stdout = sys.stdout
sys.stdout= open("cls_svm_pca_64.txt","w")

for i in [0,1,2,3,4]:
    print(f"**************fold {i}*************************")
    x_train,y_train = gu.get_train_data(i)
    train_embed = gu.get_bert_embedding(x_train)
    print(len(train_embed))
    pca = PCA(n_components=64)
    X_PCA = pca.fit_transform(train_embed)
    print(X_PCA.shape)
    clf = svm.SVC(kernel='rbf',gamma=0.5)
    #clf = svm.LinearSVC()
    clf.fit(X_PCA,y_train)

    print("Validation....")
    x_val,y_val = gu.get_val_data(i)
    val_embed = gu.get_bert_embedding(x_val)

    X_PCA_val = pca.fit_transform(val_embed)

    res = clf.predict(X_PCA_val)

    print(classification_report(y_val,res))

In [ ]:
#this code uses svm on tsne
import sys
old_stdout = sys.stdout
sys.stdout= open("cls_svm_pca_64.txt","w")

for i in [0,1,2,3,4]:
    print(f"**************fold {i}*************************")
    x_train,y_train = gu.get_train_data(i)
    train_embed = gu.get_bert_embedding(x_train)
    print(len(train_embed))
    tsne = TSNE(n_components=64,random_state=42)
    X_tsne = tsne.fit_transform(train_embed)
    
    #pca = PCA(n_components=64)
    #X_PCA = pca.fit_transform(train_embed)
    #print(X_PCA.shape)
    clf = svm.SVC(kernel='rbf',gamma=0.5)
    #clf = svm.LinearSVC()
    clf.fit(X_tsne,y_train)

    print("Validation....")
    x_val,y_val = gu.get_val_data(i)
    val_embed = gu.get_bert_embedding(x_val)

    X_PCA_val = pca.fit_transform(val_embed)

    res = clf.predict(X_PCA_val)

    print(classification_report(y_val,res))

In [ ]:

old_stdout = sys.stdout
sys.stdout= open("cls_svm_pca_64.txt","w")

print("This experiment use tsne on bert mean with 2 components. and feed them into svm")
for i in [0,1,2,3,4]:
    print(f"**************fold {i}*************************")
    x_train,y_train = gu.get_train_data(i)
    x_train = x_train[:100]
    y_train = y_train[:100]
    train_embed = gu.get_bert_embedding(x_train)
    print(len(train_embed))
    tsne = TSNE(n_components=2,random_state=42)#,method='exact')
    X_tsne = tsne.fit_transform(train_embed)
    print("tsne is done")
 
    clf = svm.SVC(kernel='rbf',gamma=0.5)
    clf.fit(X_tsne,y_train)

    print("Validation....")
    x_val,y_val = gu.get_val_data(i)
    x_val = x_val[:100]
    y_val = y_val[:100]
    val_embed = gu.get_bert_embedding(x_val)
    X_tsne_val = tsne.fit_transform(val_embed)

    res = clf.predict(X_tsne_val)

    print(classification_report(y_val,res))

In [ ]:
#this code uses svm on tsne
old_stdout = sys.stdout
sys.stdout= open("cls_svm_tsn_pca_3.txt","w")

print("This experiment use tsne on bert mean with 3 components. and feed them into svm")
for i in [0]:#,1,2,3,4]:
    print(f"**************fold {i}*************************")
    x_train,y_train = gu.get_train_data(i)
    train_embed = gu.get_bert_embedding(x_train)
    print(len(train_embed))
    tsne = TSNE(n_components=3,random_state=42)#,method='exact')
    X_tsne = tsne.fit_transform(train_embed)

    clf = svm.SVC(kernel='rbf',gamma=0.5)
    clf.fit(X_tsne,y_train)

    print("Validation....")
    x_val,y_val = gu.get_val_data(i)
    val_embed = gu.get_bert_embedding(x_val)
    X_tsne_val = tsne.fit_transform(val_embed)
    #X_PCA_val = pca.fit_transform(val_embed)

    res = clf.predict(X_tsne_val)

    print(classification_report(y_val,res))